### Error Mitigation Workflow with mitiq on Amazon Braket

In this notebook, we will show how we can use the previous three methodologies, namely readout error mitigation, Pauli twirling, and Zero-noise extrapolation, in conjunction with ProgramSets in an applied molecular example. 

The rationale is that while ZNE will extrapolate over a continuous noise parameter, measurement errors are often orders of magnitude larger, and also only occur once, so can be corrected before mitigation begins. 

To coordinate our error mitigation pipeline, we will utilize some prebuilt functions for distributing circuits to ProgramSets in the `tools` folder.

In [ ]:
import sys
import os
import numpy as np
from braket.circuits import Circuit
from braket.parametric import FreeParameter
np.set_printoptions(precision=3, linewidth=200, suppress=True)
from noise_models import qd_depol

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir))) # parent  
from tools.observable_tools import pauli_grouping
from tools.program_set_tools import run_with_program_sets
from mitiq.zne import construct_circuits, combine_results, RichardsonFactory
from mitiq.pt import generate_pauli_twirl_variants


In [ ]:
def ising_hamiltonian(
        hopping : float, 
        self_interaction : float, 
        num_qubits : int):
    hamiltonian = []
    n = num_qubits
    for i in range(num_qubits):
        hamiltonian.append(
            (self_interaction,i*"I"+"Z"+(n-i-1)*"I")
            )
        if i>0:
            hamiltonian.append(
                (hopping, (i-1)*"I"+"XX"+(n-i-1)*"I"))
    return hamiltonian



[(0.3, 'ZIIIIIIIII'),
 (0.3, 'IZIIIIIIII'),
 (0.5, 'XXIIIIIIII'),
 (0.3, 'IIZIIIIIII'),
 (0.5, 'IXXIIIIIII'),
 (0.3, 'IIIZIIIIII'),
 (0.5, 'IIXXIIIIII'),
 (0.3, 'IIIIZIIIII'),
 (0.5, 'IIIXXIIIII'),
 (0.3, 'IIIIIZIIII'),
 (0.5, 'IIIIXXIIII'),
 (0.3, 'IIIIIIZIII'),
 (0.5, 'IIIIIXXIII'),
 (0.3, 'IIIIIIIZII'),
 (0.5, 'IIIIIIXXII'),
 (0.3, 'IIIIIIIIZI'),
 (0.5, 'IIIIIIIXXI'),
 (0.3, 'IIIIIIIIIZ'),
 (0.5, 'IIIIIIIIXX')]

In [ ]:
bases, pauli_terms = pauli_grouping(ising_hamiltonian(0.5,1,6))
print(bases)
print(f'number of distinct circuits: {len(bases)}') 


['ZZZZZZZZZZ', 'XXXXXXXXXX']
number of distinct circuits: 2


In [ ]:

def test_circuit(
        theta1 : FreeParameter,
        theta2 : FreeParameter,
        num_qubits : int):
    circ = Circuit()
    for i in range(num_qubits):
        circ.rx(i,theta1)
    for i in range(0,num_qubits-1,2):
        circ.cz(i,i+1)
    for i in range(num_qubits):
        circ.rx(i,theta2)        
    for i in range(1,num_qubits-1,2):
        circ.cz(i,i+1)
    return circ

alp = FreeParameter("alp")
bet = FreeParameter("bet")

ansatz = test_circuit(alp, bet, 6)

Now let's construct our pipeline. We will use essentially two steps, in the first, we characterize our readout error. In the second, we will combine observable measurement, twirling, and zero-noise extrapolation together. 



In [ ]:

scale_factors = [1,5,9]
num_twirls = 20

circ = ansatz.make_bound_circuit({"alp":0.0, "bet":0.0})
circuits = np.array([generate_pauli_twirl_variants(c) for c in construct_circuits(circ, scale_factors=scale_factors)])
print(circuits.shape)
print(circ)


In [ ]:

test = run_with_program_sets(
    circuits, bases, pauli_terms, parameters = [{}], device = qd_depol,
    shots_per_executable=100)

# the output of test is a dim_circ + dim_para + dim_bases, which is (3,10), (1,) and (2,) respectively

print(test.shape)

# first sum over the observables 
twirled = np.sum(test, axis=(3))
print(twirled)
# then, average over the twirls 
zne_results = np.average(twirled, axis = (1,2))
print(zne_results)
mitigated = combine_results(scale_factors=scale_factors, results = zne_results, extrapolation_method=RichardsonFactory.extrapolate)
# extra_tail = combine_results(scale_factors=scale_factors + [100], results = zne_results.tolist() + [0,], extrapolation_method=RichardsonFactory.extrapolate)

print('noisy_result: ',)
print(zne_results[0])
print('mitigated result: ',)
print(mitigated)
# print('extra with 0 added: ',)
# print(extra_tail)
print('Improvement: ')
delta= abs(zne_results[0] - 6.0)
rel_err = abs(mitigated - 6.0)
print(f"{(1 - rel_err / delta) * 100 :.2f}% reduction in error")
print(f"{delta / rel_err :.1f}x improvement")